# &AElig;co
***Calc module***
<br/><br/>

✈️♻️

This code is part of the Master's Thesis "A Life Cycle Approach for Aircraft Ecodesign", by Giácomo Parolin.

``Please enter the following information:``

In [37]:
import os
os.chdir("C:/Users/giparoli/Documents/Projetos/AEco")

input_path = './Data/Iris_inputs.xlsx'
input_sheet = 'Iris'
output_path = './Outputs/'+ input_sheet + '_outputs'
database_path = './Data/database_Iris.xlsx'

iterations = 10000
chunks = 100  #'auto' or more than 500

In [38]:
from Tools_Iris import *
from Model_Iris import *

# LCI

The aircraft's life cycle was divided into four parts:
1. Development and Engineering
2. Manufacturing and Assembly
3. Operation
4. End-of-Life

First, we execute function to read inputs from designated `input_path` and transform it into Parameters and ParameterSet object `p`.

In [39]:
inputs = read_inputs(input_path, input_sheet)

In [40]:
p = ParameterSet(inputs, iterations, chunks)

Samples are stored in a xr.Dataset object accessible throught the `p.data` ParameterSet attribute.

Then, we run the function that calculates the functional unit for the study.

In [41]:
p = func_unit(p)

In [42]:
p.data["ha_fleet"].mean().compute()

<xarray.DataArray 'ha_fleet' ()>
array(2.40462823e+09)

In [43]:
p.data["ha_life"].mean().compute()

<xarray.DataArray 'ha_life' ()>
array(5716338.57123252)

<br></br>
Executing the UP reading and writing functions.

In [44]:
UP_dataframe = read_unit_processes(database_path)
UP = unit_process_dataset(UP_dataframe)

Building the empty LCI to start the calculation.

In [45]:
inventory = LCI(name=input_sheet, iterations=iterations, UP=UP, parameters=p)
inventory.run()

<xarray.Dataset>
Dimensions:         (Substances: 1835, i: 10000)
Coordinates:
    Units           (Substances) object 'kg' 'kg' 'kg' 'kg' ... 'kg' 'kBq' 'kBq'
  * Substances      (Substances) MultiIndex
  - name            (Substances) object '1,4-Butanediol' ... 'Zirconium-95'
  - compartment     (Substances) object 'Water' 'Air' 'Air' ... 'Water' 'Air'
  - subcompartment  (Substances) object 'surface water' ... 'non-urban air or...
  * i               (i) int32 0 1 2 3 4 5 6 ... 9994 9995 9996 9997 9998 9999
Data variables: (12/16)
    Office          (Substances, i) float64 dask.array<chunksize=(1835, 100), meta=np.ndarray>
    Infrastructure  (Substances, i) float64 dask.array<chunksize=(1835, 100), meta=np.ndarray>
    Capital         (Substances, i) float64 dask.array<chunksize=(1835, 100), meta=np.ndarray>
    Materials       (Substances, i) float64 dask.array<chunksize=(1835, 100), meta=np.ndarray>
    Factory         (Substances, i) float64 dask.array<chunksize=(1835, 100), meta=np.ndarray>
    Logistics       (Substances, i) float64 dask.array<chunksize=(1835, 100), meta=np.ndarray>
    ...              ...
    Fuel            (Substances, i) float64 dask.array<chunksize=(1835, 100), meta=np.ndarray>
    Recycling       (Substances, i) float64 dask.array<chunksize=(1835, 100), meta=np.ndarray>
    Incineration    (Substances, i) float64 dask.array<chunksize=(1835, 100), meta=np.ndarray>
    Landfill        (Substances, i) float64 dask.array<chunksize=(1835, 100), meta=np.ndarray>
    Prototypes      (Substances, i) float64 dask.array<chunksize=(1835, 100), meta=np.ndarray>
    Certification   (Substances, i) float64 dask.array<chunksize=(1835, 100), meta=np.ndarray>
Attributes:
    Name:     Iris

The LCI data can be accessed by the `LCI.data` attribute.

# LCIA

Extracting the characterization factors from the excel file located in `database_path`.

Creating an instance of the characterization factors class:

In [46]:
MP_data, EP_data = read_CF(database_path)
CF = CharactFactors(MP_data, EP_data, UP.Substances)

Adjusting the NLT category CFs.

In [47]:
CF.MP.dataset['NLT'].data = np.negative(CF.MP.dataset['NLT'].data)
CF.MP.to_array();

The impacts CFs can be accessed by the `CharactFactors.Midpoint.dataset` or `CharactFactors.Endpoint.dataset` attributes.

In [48]:
display(CF.MP.dataset)
display(CF.EP.dataset)

<xarray.Dataset>
Dimensions:         (Substances: 1835)
Coordinates:
  * Substances      (Substances) MultiIndex
  - name            (Substances) object '1,4-Butanediol' ... 'Zirconium-95'
  - compartment     (Substances) object 'Water' 'Air' 'Air' ... 'Water' 'Air'
  - subcompartment  (Substances) object 'surface water' ... 'non-urban air or...
Data variables: (12/18)
    FE              (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    MRD             (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    POF             (Substances) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    MET             (Substances) float64 0.0 0.0 0.000283 ... 0.0 0.0 0.0
    TA              (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    ULO             (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    ...              ...
    ALO             (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    HT              (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    IR              (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    FD              (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    WD              (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    CC              (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0

<xarray.Dataset>
Dimensions:     (Categories: 18)
Coordinates:
  * Categories  (Categories) object 'CC' 'OD' 'TA' 'FE' ... 'WD' 'MRD' 'FD'
Data variables:
    HH          (Categories) float64 1.4e-06 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    ED          (Categories) float64 7.93e-09 0.0 5.8e-09 ... 0.0 0.0 0.0
    RA          (Categories) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0715 0.165

<br></br>
**Executing the Impact Assessment.**

In [49]:
aeco = LCIA.build(inventory, CF)

## EOL test

In [50]:
E_sort_constant = 0.4645 / 3.6  # kWh/kg of material, on average
inventory.p["E_sort"] = E_sort_constant * inventory.p['OEW']
LCI_sort = inventory.electricity(inventory.p["E_sort"])

In [51]:
import xarray as xr
import dask.array as da

In [52]:
materials = ['Al','steel','Ti','inconel','GFRP','CFRP']
scenarios = ['ldf', 'incin','recycl']
#chunks = self.data.chunks['i'][0]
iterations = inventory.data.i.size
UP_eol = inventory.UP.rename_vars({'Landfill':'ldf','Incineration':'incin','Aluminium':'Al',
                                'Titanium':'Ti', 'Inconel':'inconel','Steel':'steel'})
eol = xr.Dataset({scenario: (['Substances','i'],da.empty((1835,iterations), chunks="auto"))
                        for scenario in scenarios}, coords=inventory.data.coords)
        #chunks=(1835,chunks)


In [53]:
for scenario in scenarios:
    for material in materials:
        inventory.p[scenario+"_"+material] = inventory.p["p_"+scenario+"_"+material]*inventory.p[material]
        if scenario == 'recycl':
            eol[scenario] += UP_eol[material] * inventory.p[scenario + "_" + material]
        else:
            eol[scenario] += UP_eol[scenario] * inventory.p[scenario + "_" + material]

In [54]:
inventory.p["p_recycl_Al"].compute()

<xarray.DataArray 'p_recycl_Al' (i: 10000)>
array([0.4398284 , 0.48384681, 0.43892184, ..., 0.56475381, 0.44623402,
       0.47598474])
Coordinates:
  * i        (i) int32 0 1 2 3 4 5 6 7 ... 9993 9994 9995 9996 9997 9998 9999
Attributes:
    unit:         nan
    description:  percentage of recycled aluminum
    max:          0.6
    min:          0.4
    exp:          0.5
    alpha:        3.0
    beta:         3.0
    mean:         0.5

In [55]:
inventory.p["Al"].compute()

<xarray.DataArray 'Al' (i: 10000)>
array([1324.91934143, 1905.43900536, 1692.68397517, ..., 1854.69299814,
       1474.16737409, 1614.34958067])
Coordinates:
  * i        (i) int32 0 1 2 3 4 5 6 7 ... 9993 9994 9995 9996 9997 9998 9999

In [56]:
inventory.p["recycl_Al"].compute()

<xarray.DataArray 'recycl_Al' (i: 10000)>
array([ 582.73715758,  921.94058066,  742.95596813, ..., 1047.444946  ,
        657.82364039,  768.40576989])
Coordinates:
  * i        (i) int32 0 1 2 3 4 5 6 7 ... 9993 9994 9995 9996 9997 9998 9999

In [57]:
test_rec = (LCI_sort - eol['recycl']) / inventory.p["ha_life"]

In [62]:
inventory.data["Materials"].compute()

<xarray.DataArray 'Materials' (Substances: 1835, i: 10000)>
array([[ 1.12697830e-14,  2.13650593e-14,  2.40861332e-14, ...,
         2.09552768e-14,  1.33463976e-14,  1.28652919e-14],
       [ 4.90002925e-15,  9.28935677e-15,  1.04724068e-14, ...,
         9.11116065e-15,  5.80291949e-15,  5.59372544e-15],
       [-4.39887346e-16,  2.83489994e-16,  1.45355320e-15, ...,
         8.52235400e-16, -2.60918625e-16,  8.47444044e-17],
       ...,
       [-4.33508069e-12, -7.43026857e-13, -2.07238058e-12, ...,
        -3.42521221e-13, -2.09057600e-12, -8.63694717e-13],
       [ 1.69140034e-10,  2.38764721e-10,  1.81731327e-10, ...,
         1.80540686e-10,  1.66928125e-10,  1.33709797e-10],
       [ 1.40735187e-12,  9.24363224e-13, -5.36471542e-13, ...,
        -2.77437463e-13,  6.82279494e-13,  8.24998232e-14]])
Coordinates:
    Units           (Substances) object 'kg' 'kg' 'kg' 'kg' ... 'kg' 'kBq' 'kBq'
  * Substances      (Substances) MultiIndex
  - name            (Substances) object '1,4-Butanediol' ... 'Zirconium-95'
  - compartment     (Substances) object 'Water' 'Air' 'Air' ... 'Water' 'Air'
  - subcompartment  (Substances) object 'surface water' ... 'non-urban air or...
  * i               (i) int32 0 1 2 3 4 5 6 ... 9994 9995 9996 9997 9998 9999

In [58]:
test_rec.compute()

<xarray.DataArray (Substances: 1835, i: 10000)>
array([[-4.92757716e-07, -6.51655322e-07, -5.28303799e-07, ...,
        -5.25978026e-07, -5.30125565e-07, -4.22415012e-07],
       [-4.92757661e-07, -6.51655247e-07, -5.28303716e-07, ...,
        -5.25977955e-07, -5.30125510e-07, -4.22414963e-07],
       [-4.92757644e-07, -6.51655224e-07, -5.28303690e-07, ...,
        -5.25977933e-07, -5.30125494e-07, -4.22414948e-07],
       ...,
       [-4.92759101e-07, -6.51659332e-07, -5.28305799e-07, ...,
        -5.25981816e-07, -5.30127635e-07, -4.22417010e-07],
       [-4.92775891e-07, -6.51697290e-07, -5.28324533e-07, ...,
        -5.26013819e-07, -5.30146430e-07, -4.22435616e-07],
       [-4.92755713e-07, -6.51652471e-07, -5.28300525e-07, ...,
        -5.25975177e-07, -5.30123239e-07, -4.22413132e-07]])
Coordinates:
  * Substances      (Substances) MultiIndex
  - name            (Substances) object '1,4-Butanediol' ... 'Zirconium-95'
  - compartment     (Substances) object 'Water' 'Air' 'Air' ... 'Water' 'Air'
  - subcompartment  (Substances) object 'surface water' ... 'non-urban air or...
    Units           (Substances) object 'kg' 'kg' 'kg' 'kg' ... 'kg' 'kBq' 'kBq'
  * i               (i) int32 0 1 2 3 4 5 6 ... 9994 9995 9996 9997 9998 9999

In [ ]:

self.data["Incineration"] = eol["incin"] / self.p["ha_life"]
self.data["Landfill"] = eol["ldf"] / self.p["ha_life"]

## Midpoint Results

Raw midpoint results can be accessed by the `LCIA.MP` attribute. Mean midpoint results are calculated by the `LCIA.mean` method, with pathway set to "MP".

In [ ]:
display(aeco.MP)

<xarray.Dataset>
Dimensions:         (Categories: 18, i: 10000)
Coordinates:
  * i               (i) int32 0 1 2 3 4 5 6 ... 9994 9995 9996 9997 9998 9999
  * Categories      (Categories) <U3 'FE' 'MRD' 'POF' 'MET' ... 'FD' 'WD' 'CC'
Data variables: (12/16)
    Office          (i, Categories) float64 dask.array<chunksize=(100, 18), meta=np.ndarray>
    Infrastructure  (i, Categories) float64 dask.array<chunksize=(100, 18), meta=np.ndarray>
    Capital         (i, Categories) float64 dask.array<chunksize=(100, 18), meta=np.ndarray>
    Materials       (i, Categories) float64 dask.array<chunksize=(100, 18), meta=np.ndarray>
    Factory         (i, Categories) float64 dask.array<chunksize=(100, 18), meta=np.ndarray>
    Logistics       (i, Categories) float64 dask.array<chunksize=(100, 18), meta=np.ndarray>
    ...              ...
    Fuel            (i, Categories) float64 dask.array<chunksize=(100, 18), meta=np.ndarray>
    Recycling       (i, Categories) float64 dask.array<chunksize=(100, 18), meta=np.ndarray>
    Incineration    (i, Categories) float64 dask.array<chunksize=(100, 18), meta=np.ndarray>
    Landfill        (i, Categories) float64 dask.array<chunksize=(100, 18), meta=np.ndarray>
    Prototypes      (i, Categories) float64 dask.array<chunksize=(100, 18), meta=np.ndarray>
    Certification   (i, Categories) float64 dask.array<chunksize=(100, 18), meta=np.ndarray>
Attributes:
    Name:     Iris

In [31]:
aeco.mean("MP", by="sum").compute()

<xarray.DataArray 'stack-03b05676f6cd416f06cb8b7a5bde1b3e' (Categories: 18)>
array([3.72687101e-02, 8.95412978e+00, 2.21554975e-01, 1.72921881e+01,
       3.87553336e-01, 3.69986159e-01, 1.71938833e-01, 6.74103030e-02,
       1.00649784e+00, 2.80998828e-03, 1.14525497e-04, 2.71099227e+00,
       8.91167127e+00, 2.66357171e+02, 7.68221900e-01, 2.48229814e+01,
       5.22942701e+01, 5.86013866e+01])
Coordinates:
  * Categories  (Categories) <U3 'FE' 'MRD' 'POF' 'MET' ... 'IR' 'FD' 'WD' 'CC'

## Endpoint Results

Raw endpoint results can be accessed by the `LCIA.EP` attribute. Mean endpoint results are calculated by the `LCIA.mean` method, with pathway set to "EP".

In [29]:
display(aeco.EP)

<xarray.Dataset>
Dimensions:         (AOP: 3, i: 10000)
Coordinates:
  * i               (i) int32 0 1 2 3 4 5 6 ... 9994 9995 9996 9997 9998 9999
  * AOP             (AOP) <U2 'HH' 'ED' 'RA'
Data variables: (12/16)
    Office          (i, AOP) float64 dask.array<chunksize=(100, 3), meta=np.ndarray>
    Infrastructure  (i, AOP) float64 dask.array<chunksize=(100, 3), meta=np.ndarray>
    Capital         (i, AOP) float64 dask.array<chunksize=(100, 3), meta=np.ndarray>
    Materials       (i, AOP) float64 dask.array<chunksize=(100, 3), meta=np.ndarray>
    Factory         (i, AOP) float64 dask.array<chunksize=(100, 3), meta=np.ndarray>
    Logistics       (i, AOP) float64 dask.array<chunksize=(100, 3), meta=np.ndarray>
    ...              ...
    Fuel            (i, AOP) float64 dask.array<chunksize=(100, 3), meta=np.ndarray>
    Recycling       (i, AOP) float64 dask.array<chunksize=(100, 3), meta=np.ndarray>
    Incineration    (i, AOP) float64 dask.array<chunksize=(100, 3), meta=np.ndarray>
    Landfill        (i, AOP) float64 dask.array<chunksize=(100, 3), meta=np.ndarray>
    Prototypes      (i, AOP) float64 dask.array<chunksize=(100, 3), meta=np.ndarray>
    Certification   (i, AOP) float64 dask.array<chunksize=(100, 3), meta=np.ndarray>
Attributes:
    Name:     Iris

In [30]:
aeco.mean("EP", by="sum").compute()

<xarray.DataArray 'stack-f71240e967d31033ff3e49cc75ae9c3f' (AOP: 3)>
array([3.24910064e-04, 4.82882070e-07, 4.73667566e+00])
Coordinates:
  * AOP      (AOP) <U2 'HH' 'ED' 'RA'

## Contribution to Variance (CTV)

Calculate CTV results for all inputs with midpoint and endpoint results. CTV results can be acessed by the `LCIA.CTV` attribute.

In [32]:
aeco.build_CTV(parameterset=p)

<xarray.Dataset>
Dimensions:     (Parameters: 79)
Coordinates:
  * Parameters  (Parameters) <U18 'takt' 'OEW' ... 'p_recycl_CFRP'
Data variables: (12/21)
    FE          (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    MRD         (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    POF         (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    MET         (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    TA          (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    ULO         (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    ...          ...
    FD          (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    WD          (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    CC          (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    HH          (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    ED          (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    RA          (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>

In [33]:
aeco.CTV["CC"].sortby(aeco.CTV["CC"], ascending=False).compute()

c:\Users\giparoli\Documents\Projetos\AEco\Scripts\Tools_Iris.py:307: RuntimeWarning: invalid value encountered in true_divide
  return covariance_gufunc(x, y) / (x.std(axis=-1) * y.std(axis=-1))
c:\Users\giparoli\Documents\Projetos\AEco\Scripts\Tools_Iris.py:307: RuntimeWarning: invalid value encountered in true_divide
  return covariance_gufunc(x, y) / (x.std(axis=-1) * y.std(axis=-1))


<xarray.DataArray 'CC' (Parameters: 79)>
array([4.74700312e+01, 4.62697815e+01, 5.07057249e-01, 4.87346991e-01,
       4.33333430e-01, 3.49281619e-01, 3.19182379e-01, 2.69330639e-01,
       2.52302135e-01, 2.39955679e-01, 2.14533371e-01, 2.06010626e-01,
       2.04265740e-01, 1.66513210e-01, 1.42117510e-01, 1.30530735e-01,
       1.28504147e-01, 1.23071050e-01, 1.15390738e-01, 1.11207290e-01,
       1.10118561e-01, 1.06788367e-01, 8.22800499e-02, 7.88428332e-02,
       7.76600250e-02, 7.60684684e-02, 7.52822346e-02, 7.34410978e-02,
       7.19403975e-02, 6.91666649e-02, 6.63588427e-02, 6.62959633e-02,
       6.59380906e-02, 6.47732407e-02, 6.04603873e-02, 5.98208340e-02,
       5.68925128e-02, 5.30753209e-02, 5.11435760e-02, 4.84349032e-02,
       4.63235245e-02, 4.06947694e-02, 3.64932148e-02, 3.38122507e-02,
       3.36309143e-02, 3.04238261e-02, 2.99538973e-02, 2.88054258e-02,
       2.43074884e-02, 2.40510190e-02, 2.20773802e-02, 1.28497455e-02,
       1.27764752e-02, 1.15880548e-02, 1.15828238e-02, 9.55418832e-03,
       8.92365300e-03, 6.98779068e-03, 6.61555150e-03, 4.28843411e-03,
       2.08686641e-03, 1.92427803e-03, 1.91129555e-03, 1.56411630e-03,
       8.09200214e-04, 5.81795146e-04, 4.02690469e-04, 3.02799706e-04,
       1.47013154e-04, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00])
Coordinates:
  * Parameters  (Parameters) <U18 'pesticide_eff' ... 'ironbirds'

# Export Results

Export MP, EP, LCI and CTV result to NetCDF file.

In [34]:
aeco.save(output_path, LCI=False)

c:\Users\giparoli\Documents\Projetos\AEco\Scripts\Tools_Iris.py:307: RuntimeWarning: invalid value encountered in true_divide
  return covariance_gufunc(x, y) / (x.std(axis=-1) * y.std(axis=-1))


LCIA saved at ./Outputs/Iris_outputs


In [36]:
aeco.save(('./Local/'+ input_sheet + '_outputs with LCI'), LCI=True)

c:\Users\giparoli\Documents\Projetos\AEco\Scripts\Tools_Iris.py:307: RuntimeWarning: invalid value encountered in true_divide
  return covariance_gufunc(x, y) / (x.std(axis=-1) * y.std(axis=-1))


LCIA saved at ./Local/Iris_outputs with LCI


In [22]:
#aeco.to_excel(output_path)